In [3]:
from openai import OpenAI


In [4]:
client = OpenAI()

In [8]:
client.files.create(file=open('fine_tuning_data.jsonl', 'rb'), purpose='fine-tune')

FileObject(id='file-VEryUbLeXSOKDjKtivZjND80', bytes=1605785, created_at=1722618995, filename='fine_tuning_data.jsonl', object='file', purpose='fine-tune', status='processed', status_details=None)

In [10]:
client.files.create(file=open('fine_tuning_data_val.jsonl', 'rb'), purpose='fine-tune')

FileObject(id='file-C1C4JPvdGySa5nCtoJSDukic', bytes=101434, created_at=1722619062, filename='fine_tuning_data_val.jsonl', object='file', purpose='fine-tune', status='processed', status_details=None)

# Fine-tuning training

In [11]:
result = client.fine_tuning.jobs.create(
    training_file='file-VEryUbLeXSOKDjKtivZjND80',
    model='gpt-3.5-turbo',
    hyperparameters={'n_epochs': 1}, # Number of times to pass over the training data
    validation_file='file-C1C4JPvdGySa5nCtoJSDukic'
)

In [12]:
result

FineTuningJob(id='ftjob-q3o9maNpVh87qQzLIAGf7vcO', created_at=1722619357, error=Error(code=None, message=None, param=None), fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(n_epochs=1, batch_size='auto', learning_rate_multiplier='auto'), model='gpt-3.5-turbo-0125', object='fine_tuning.job', organization_id='org-skBw4GYASyPZtlACUZX6o2Nb', result_files=[], seed=1693561264, status='validating_files', trained_tokens=None, training_file='file-VEryUbLeXSOKDjKtivZjND80', validation_file='file-C1C4JPvdGySa5nCtoJSDukic', estimated_finish=None, integrations=[], user_provided_suffix=None)

In [15]:
client.fine_tuning.jobs.list()

SyncCursorPage[FineTuningJob](data=[FineTuningJob(id='ftjob-q3o9maNpVh87qQzLIAGf7vcO', created_at=1722619357, error=Error(code=None, message=None, param=None), fine_tuned_model='ft:gpt-3.5-turbo-0125:personal::9rqcNd5w', finished_at=1722620673, hyperparameters=Hyperparameters(n_epochs=1, batch_size=1, learning_rate_multiplier=2), model='gpt-3.5-turbo-0125', object='fine_tuning.job', organization_id='org-skBw4GYASyPZtlACUZX6o2Nb', result_files=['file-9ML4EJxt168BSA57TCxAhSUW'], seed=1693561264, status='succeeded', trained_tokens=376193, training_file='file-VEryUbLeXSOKDjKtivZjND80', validation_file='file-C1C4JPvdGySa5nCtoJSDukic', estimated_finish=None, integrations=[], user_provided_suffix=None)], object='list', has_more=False)

In [17]:
# Losses visualization
train_event = client.fine_tuning.jobs.list_events(fine_tuning_job_id='ftjob-q3o9maNpVh87qQzLIAGf7vcO')

In [34]:
data = [event.data for event in train_event.data if len(event.data) > 0]

{'step': 525,
 'train_loss': 2.5175418853759766,
 'total_steps': 525,
 'full_valid_loss': 0.4708685133192274,
 'train_mean_token_accuracy': 0.75,
 'full_valid_mean_token_accuracy': 0.9333333333333333}

In [42]:
[item['train_loss'] for item in data]

[2.5175418853759766,
 2.384185791015625e-06,
 3.7259647846221924,
 4.76837158203125e-07,
 1.9073486328125e-06,
 2.384185791015625e-07,
 1.9073486328125e-06,
 3.814697322468419e-07,
 2.28881845032447e-06,
 4.76837158203125e-07,
 6.35782896551973e-07,
 2.738797187805176,
 3.178914482759865e-07,
 7.152557373046875e-06,
 0.8430123329162598,
 3.814697322468419e-07,
 3.178914482759865e-07,
 3.859421730041504]